In [1]:
import pandas as pd
import numpy as np
import string
import os
from sklearn.preprocessing import StandardScaler

# 1. 데이터 불러오기
df = pd.read_csv('/Users/igangsan/Desktop/Lending-Club-Project/프로젝트/lending_club_2020_train.csv')

# 2. 삭제할 변수 리스트 (84개)
drop_cols = [  # 이전에 지정한 84개
    'hardship_loan_status', 'hardship_type', 'hardship_reason', 'hardship_status',
    'deferral_term', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date',
    'hardship_length', 'hardship_dpd', 'orig_projected_additional_accrued_interest',
    'hardship_amount', 'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
    'sec_app_revol_util', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc',
    'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
    'sec_app_collections_12_mths_ex_med', 'annual_inc_joint', 'dti_joint', 'mths_since_last_record',
    'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'next_pymnt_d', 'inq_fi',
    'total_cu_tl', 'emp_title', 'num_actv_bc_tl', 'hardship_flag', 'title', 'last_pymnt_d',
    'collection_recovery_fee', 'earliest_cr_line', 'funded_amnt', 'funded_amnt_inv', 'id',
    'initial_list_status', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d',
    'last_pymnt_amnt', 'loan_amnt', 'loan_status', 'out_prncp', 'out_prncp_inv', 'policy_code',
    'pymnt_plan', 'recoveries', 'term', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
    'total_rec_late_fee', 'total_rec_prncp', 'url', 'zip_code', 'debt_settlement_flag',
    'desc', 'member_id', 'verified_status_joint', 'sec_app_mths_since_last_major_derog',
    'disbursement_method', 'debt_settlement_flag_date', 'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage', 'settlement_term'
]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

# 3. 수치형 결측치 처리 + 결측 마스크 (int)
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
missing_cols = {}
for col in num_cols:
    missing_cols[col + '_missing'] = df[col].isnull().astype(int)
    df[col] = df[col].fillna(0)
df = pd.concat([df, pd.DataFrame(missing_cols)], axis=1)

# 4. 범주형 결측치 처리
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    df[col] = df[col].fillna('Missing')

# 5. emp_length 전처리
def process_emp_length(x):
    if pd.isna(x):
        return None
    elif '< 1' in x:
        return 0.5
    elif '10+' in x:
        return 10.0
    else:
        extracted = pd.to_numeric(pd.Series(x).str.extract(r'(\d+)')[0], errors='coerce')
        return extracted.iloc[0]
if 'emp_length' in df.columns:
    df['emp_length'] = df['emp_length'].apply(process_emp_length)

# 6. revol_util 전처리
def preprocess_revol_util(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip().replace('%', '')
    try:
        return float(x)
    except:
        return np.nan
if 'revol_util' in df.columns:
    df['revol_util'] = df['revol_util'].apply(preprocess_revol_util)
    df['revol_util_missing'] = df['revol_util'].isnull().astype(int)
    df['revol_util'] = df['revol_util'].fillna(0)

# 7. grade / sub_grade 수치화
grade_map = {k: v for v, k in enumerate(['A','B','C','D','E','F','G'], start=1)}
if 'grade' in df.columns:
    df['grade_num'] = df['grade'].map(grade_map)
if 'sub_grade' in df.columns:
    sub_grades = [f"{l}{n}" for l in string.ascii_uppercase[:7] for n in range(1, 6)]
    sub_grade_map = {k: v for v, k in enumerate(sub_grades, start=1)}
    df['sub_grade_num'] = df['sub_grade'].map(sub_grade_map)

# 8. 더미 인코딩 (int형)
dummy_cols = ['addr_state', 'home_ownership', 'application_type', 'purpose', 'verification_status']
df = pd.get_dummies(df, columns=[col for col in dummy_cols if col in df.columns], dtype='uint8')

# 9. 이상치 클리핑
for col in num_cols:
    if col in df.columns:
        mean, std = df[col].mean(), df[col].std()
        df[col] = df[col].clip(mean - 3 * std, mean + 3 * std)

# 10. 스케일링
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

# 11. 메모리 조각화 정리
df = df.copy()

# 12. 학습 변수 추출
processed_vars = set(['emp_length', 'revol_util', 'grade', 'sub_grade'])
newly_created_vars = [col for col in df.columns if (
    col.endswith('_missing') or
    col in ['grade_num', 'sub_grade_num'] or
    any(col.startswith(prefix + '_') for prefix in ['addr_state', 'home_ownership', 'application_type', 'purpose', 'verification_status'])
)]
learnable_vars = []
for col in df.columns:
    if col in newly_created_vars:
        learnable_vars.append(col)
    elif col not in drop_cols and col not in processed_vars and col != 'DELINQUENT':
        learnable_vars.append(col)

# 13. 저장
output_path = 'loan_data_processed.csv'
df.to_csv(output_path, index=False)
file_size = os.path.getsize(output_path)

# 14. 출력
print(f"✅ 저장 완료: '{output_path}' ({file_size / (1024 * 1024):.2f} MB)")
print(f"✅ 학습에 사용할 변수 수: {len(learnable_vars)}개")
print(f"🔹 예시 변수:\n{learnable_vars[:15]} ...")



/var/folders/h1/2kvn4fr11vn6f3469636zhp40000gn/T/ipykernel_6379/278855427.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/igangsan/Desktop/Lending-Club-Project/프로젝트/lending_club_2020_train.csv')


✅ 저장 완료: 'loan_data_processed.csv' (2711.44 MB)
✅ 학습에 사용할 변수 수: 215개
🔹 예시 변수:
['annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'verification_status_joint'] ...


In [2]:
df = pd.read_csv("loan_data_processed.csv")
df.head()

,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,...,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,verification_status_Missing,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified
0,A,A5,2.0,-0.656662,-1.059234,1.207731,1.641646,1.641640,-0.671514,0.236274,...,0,0,0,0,0,0,0,0,0,1
1,C,C2,2.0,0.296617,-0.160880,-0.412485,0.439330,0.439327,1.807192,-0.720492,...,0,0,0,0,0,0,0,0,1,0
2,E,E4,10.0,0.607892,1.616299,1.207731,-0.913276,-0.913274,0.567839,-0.285598,...,0,0,0,0,0,0,0,0,0,1
3,B,B2,10.0,-0.851209,-0.413735,1.207731,-0.762987,-0.762985,0.567839,0.236274,...,0,0,0,0,0,0,0,1,0,0
4,C,C4,10.0,-0.958210,3.318443,-0.412485,-0.462407,-0.462407,-0.671514,0.323253,...,0,0,0,0,0,0,0,0,0,1


In [3]:
for i in df.columns:
    print(i)

grade
sub_grade
emp_length
annual_inc
dti
delinq_2yrs
fico_range_low
fico_range_high
inq_last_6mths
mths_since_last_delinq
open_acc
pub_rec
revol_bal
revol_util
total_acc
last_fico_range_high
last_fico_range_low
collections_12_mths_ex_med
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_act_il
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_last_12m
acc_open_past_24mths
avg_cur_bal
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_il_acct
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
mths_since_recent_inq
mths_since_recent_revol_delinq
num_accts_ever_120_pd
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_120dpd_2m
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankruptcies
tax_liens
tot_hi_cred_lim
total_